In [35]:
#import libs
import pandas as pd
from py2neo.database import Graph
import calendar
import time
# create neo4j graph connection
graph = Graph('bolt://127.0.0.1:7687', auth=(username, password), name="database")

In [3]:
#test a label
df = pd.DataFrame(graph.run("MATCH (n:Movie) UNWIND keys(n) as property RETURN labels(n) as node,id(n) as node_id , property, n[property] as value").to_table(),columns=['node','node_id','property','value'])

In [4]:
#print info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   node      113 non-null    object
 1   node_id   113 non-null    int64 
 2   property  113 non-null    object
 3   value     113 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.7+ KB


In [5]:
#print first 5 rows
df.head()

,node,node_id,property,value
0,[Movie],4,title,The Matrix
1,[Movie],4,tagline,Welcome to the Real World
2,[Movie],4,released,1999
3,[Movie],9,title,The Matrix Reloaded
4,[Movie],9,tagline,Free your mind


In [6]:
# convert object type to string
types =['str','string']
for col in list(df.columns):
    for type in types:
        df[col]=df[col].astype(type)

In [7]:
# verify datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   node      113 non-null    string
 1   node_id   113 non-null    string
 2   property  113 non-null    string
 3   value     113 non-null    string
dtypes: string(4)
memory usage: 3.7 KB


In [9]:
# remove characters from node column after converting to string
df['node'] = df['node'].str.replace(r"[\']", '')
df['node'] = df['node'].str.replace(r"[\[']", '')
df['node'] = df['node'].str.replace(r"[\]']", '')
df.head()

,node,node_id,property,value
0,Movie,4,title,The Matrix
1,Movie,4,tagline,Welcome to the Real World
2,Movie,4,released,1999
3,Movie,9,title,The Matrix Reloaded
4,Movie,9,tagline,Free your mind


In [57]:
def get_all_properties(nodename=''):
    # if no node node labels are provided, get all nodes
    ts = str(calendar.timegm(gmt))
    if len(nodename)==0:
        newnode="n"
        filename="all"
    else:
        newnode="n:"+nodename
        filename=nodename
    
    #create a node_property data frame
    node_property_df = pd.DataFrame(graph.run("MATCH ( "+newnode+") UNWIND keys(n) as property RETURN labels(n) as node,id(n) as node_id , property, n[property] as value").to_table(),columns=['node','node_id','property','value'])
    # convert object to string and remove unwanted characters
    types =['str','string']
    for col in list(node_property_df.columns):
        for type in types:
            node_property_df[col]=node_property_df[col].astype(type)
    #better way to do, but currently just using below to remove unwanted characters
    node_property_df['node'] = node_property_df['node'].str.replace(r"[\']", '')
    node_property_df['node'] = node_property_df['node'].str.replace(r"[\[']", '')
    node_property_df['node'] = node_property_df['node'].str.replace(r"[\]']", '')
    # NOTE - movie dataset in neo4j, doesn't have a "id" as a property. So, concentated node-> name and node(id) for building unique index for pivot table.
    node_property_df['node_n_id'] = node_property_df['node']+'_'+node_property_df['node_id']
    #create a pivot dataframe
    pivot = node_property_df.pivot(index='node_n_id',columns='property',values='value')
    # create a sorted pivot table
    sorted_result = pivot.sort_values('node_n_id', ascending=False)
    # both pivot and sorted_result can be one line. for readability, 2 lines of code are used. it can be modified later with the below line
    # retun (node_property_df.pivot(index='node_n_id',columns='property',values='value')).sort_values('node_n_id', ascending=False)
    # save dataframe as a csv file with timestamp
    sorted_result.to_csv(filename+"_"+ts+".csv" )
    #return dataframe
    return sorted_result
        

In [58]:
# get all property values for node Person
get_all_properties("Person")

property,born,name
node_n_id,,
Person_99,1930,Clint Eastwood
Person_98,1930,Richard Harris
Person_96,1931,Mike Nichols
Person_94,1954,Zach Grenier
Person_93,1974,Christian Bale
...,...,...
Person_103,1958,Ice-T
Person_102,1968,Dina Meyer
Person_101,1947,Takeshi Kitano


In [53]:
# get all property values for node Movie
get_all_properties("Movie")

property,released,tagline,title
node_n_id,,,
Movie_97,1992,"It's a hell of a thing, killing a man",Unforgiven
Movie_95,1996,Come as you are,The Birdcage
Movie_92,2006,Based on the extraordinary true story of one m...,RescueDawn
Movie_9,2003,Free your mind,The Matrix Reloaded
Movie_87,2000,"Pain heals, Chicks dig scars... Glory lasts fo...",The Replacements
Movie_85,1996,In every life there comes a time when that thi...,That Thing You Do
Movie_81,1998,Can two friends sleep together and still love ...,When Harry Met Sally
Movie_78,1990,"A story of love, lava and burning desire.",Joe Versus the Volcano
Movie_73,1993,"What if someone you never met, someone you nev...",Sleepless in Seattle


In [54]:
# get all property values for all the nodes
get_all_properties()

property,born,name,released,tagline,title
node_n_id,,,,,
Person_99,1930,Clint Eastwood,<NA>,<NA>,<NA>
Person_98,1930,Richard Harris,<NA>,<NA>,<NA>
Person_96,1931,Mike Nichols,<NA>,<NA>,<NA>
Person_94,1954,Zach Grenier,<NA>,<NA>,<NA>
Person_93,1974,Christian Bale,<NA>,<NA>,<NA>
...,...,...,...,...,...
Movie_111,<NA>,<NA>,2006,Break The Codes,The Da Vinci Code
Movie_11,<NA>,<NA>,1997,Evil has its winning ways,The Devil's Advocate
Movie_105,<NA>,<NA>,2012,Everything is connected,Cloud Atlas
